## Optimization with Python - An Equitable Distribution Model
2020 February, Sabi Horvat @tourofdata
https://github.com/wpbSabi/supply_chain_models

This simple example of an equitable distribution model can be expanded for situations where demand exceeds the supply.  Therefore, all supply would be sent, with some requirements and decisions on where to send the supply.

This model utilizes pyomo, rather than PuLP, as pyomo can be expanded for non-linear problems. 
> If the criteria would be allocate supply based on quantities, those constraints would likely remain linear.
> However, if the criteria would utilize a function such as the distance formula to minimize transportation costs, then the model would be non-linear.  There are ways to "linearize" such formulations by providing the cost of transportation lanes rather than the specific distances, but this simplification may not be applicable for all models.

In [1]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory

In [2]:
origin_nodes = [0,1,2,3]
destination_nodes = [0,1,2,3,4,5,6]
supply_at_origin = [16765,93590,419425,39200]

In [3]:
# model
model = ConcreteModel()

# decision variable - amount allocated from supply_node to demand_node
model.x = Var(((i,j) for i in origin_nodes for j in destination_nodes),
              within=NonNegativeReals,
              initialize=0) 

In [4]:
# objective function: maximize distribution
def obj_rule(m):
#    sum(m.x[i,j] for i in supply_nodes for j in demand_nodes)
    return sum(m.x[i,j] for i in origin_nodes for j in destination_nodes)
model.objective_to_max = Objective(rule=obj_rule, sense=maximize)

In [5]:
# Create a set of constraints
model.constraints = ConstraintList()  

# send at least 10 units of supply to each demand node
for j in destination_nodes:
    model.constraints.add(
        sum(model.x[i,j] for i in origin_nodes) >= 10
    )
    
# capacity is limited by supply 
for i in origin_nodes:
    model.constraints.add(
        sum(model.x[i,j] for j in destination_nodes) <= supply_at_origin[i]
    )

In [6]:
opt = SolverFactory('ipopt')  # choose a solver
results = opt.solve(model)  # solve the model with the selected solver

# print model status
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print("Optimal solution found")
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print("Infeasible")
else:
    print("Solver Status: ",  result.solver.status)

Optimal solution found


In [7]:
# format results
print('Total supply sent: ')
print('*'*40)
print(model.objective_to_max())

for v in model.component_objects(Var, active=True):
    print('\n\n(Origin Node, Destination Node): Supply sent from Origin to Destination')
    print('*'*40)
    varobject = getattr(model, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)
        
print('\n\nSupply received by Destination')
print('*'*40)
for d in destination_nodes:
    print('Destination ',d, ' : ', sum(varobject[(o,d)].value for o in origin_nodes))

Total supply sent: 
****************************************
568980.0056897899


(Origin Node, Destination Node): Supply sent from Origin to Destination
****************************************
    (0, 0) 2395.0000239536475
    (0, 1) 2395.0000239532765
    (0, 2) 2395.000023942208
    (0, 3) 2395.0000239576434
    (0, 4) 2395.0000239570286
    (0, 5) 2395.0000239427086
    (0, 6) 2395.0000239409824
    (1, 0) 13370.00013391205
    (1, 1) 13370.000133909285
    (1, 2) 13370.000133428608
    (1, 3) 13370.000133910979
    (1, 4) 13370.00013389932
    (1, 5) 13370.000133436093
    (1, 6) 13370.000133401161
    (2, 0) 59917.85774691055
    (2, 1) 59917.857747147435
    (2, 2) 59917.85773560562
    (2, 3) 59917.857745967114
    (2, 4) 59917.8577465219
    (2, 5) 59917.857735494574
    (2, 6) 59917.857736600236
    (3, 0) 5600.000056036892
    (3, 1) 5600.000056035577
    (3, 2) 5600.000055953751
    (3, 3) 5600.000056036262
    (3, 4) 5600.000056033091
    (3, 5) 5600.0000559547725
    (3, 

In [8]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 28
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.027632713317871094
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
